In [ ]:
import numpy as np
from datetime import datetime
from scipy.interpolate import griddata

from halem import BaseRoadmap, HALEM_space, HALEM_time, plot_timeseries

import matplotlib.pyplot as plt

plt.style.use("ggplot")
%matplotlib inline

In [ ]:
class flow_potentiaalveld(BaseRoadmap):
    def load(self):
        d = datetime.strptime("23/03/2019 00:00:00", "%d/%m/%Y %H:%M:%S")
        t0 = d.timestamp()
        x = np.arange(0, 1, 0.05)
        y = np.arange(0, 1, 0.05)
        t = np.arange(t0, (t0 + 30 * 60 * 30), 30 * 60)

        y, x = np.meshgrid(y, x)
        y = y.reshape(y.size)
        x = x.reshape(y.size)

        nodes = np.zeros((len(x), 2))
        nodes[:, 0] = y
        nodes[:, 1] = x

        u = []
        v = []
        for node in nodes:
            ut = 0 * t + 2 * np.cos(np.pi * (node[0]))
            vt = 0 * t - 2 * np.cos(np.pi * (node[1]))
            u.append(ut)
            v.append(vt)

        v = np.transpose(np.array(v))
        u = np.transpose(np.array(u))
        WD = u * 0 + 20
        t = t
        nodes = nodes

        return {
            "v": v,
            "u": u,
            "water_depth": WD,
            "time": t,
            "nodes": nodes,
        }


class flow_dyn_cur(BaseRoadmap):
    def load(self):
        d = datetime.strptime("23/03/2019 00:00:00", "%d/%m/%Y %H:%M:%S")
        t0 = d.timestamp()
        x = np.arange(0, 1, 0.05)
        y = np.arange(0, 1, 0.05)
        t = np.arange(t0, (t0 + 30 * 60 * 30), 30 * 60)

        y, x = np.meshgrid(y, x)
        y = y.reshape(y.size)
        x = x.reshape(y.size)

        nodes = np.zeros((len(x), 2))
        nodes[:, 0] = y
        nodes[:, 1] = x

        u = []
        v = []
        for node in nodes:
            ut = 2 * np.cos(np.pi * (node[0])) * np.cos(2 * np.pi * (t - t0) / 24000)
            vt = -2 * np.cos(np.pi * (node[1])) * np.cos(2 * np.pi * (t - t0) / 24000)
            u.append(ut)
            v.append(vt)

        v = np.transpose(np.array(v))
        u = np.transpose(np.array(u))
        WD = u * 0 + 20
        t = t
        nodes = nodes

        return {
            "v": v,
            "u": u,
            "water_depth": WD,
            "time": t,
            "nodes": nodes,
        }


class flow_wait(BaseRoadmap):
    def load(self):
        d = datetime.strptime("23/03/2019 00:00:00", "%d/%m/%Y %H:%M:%S")
        t0 = d.timestamp()
        x = np.arange(0, 1, 0.05)
        y = np.arange(0, 1, 0.05)
        t = np.arange(t0, (t0 + 30 * 60 * 30), 10 * 60)

        y, x = np.meshgrid(y, x)
        y = y.reshape(y.size)
        x = x.reshape(y.size)

        nodes = np.zeros((len(x), 2))
        nodes[:, 0] = y
        nodes[:, 1] = x

        u = []
        v = []
        for node in nodes:
            ut = 0 * t + 0
            vt = 0 * t + 0
            u.append(ut)
            v.append(vt)

        v = np.transpose(np.array(v))
        u = np.transpose(np.array(u))
        t = t
        nodes = nodes

        WD = u * 0 + 20
        for i in range(len(nodes)):
            node = nodes[i]
            if node[0] > 0.4 and node[0] < 0.6:
                if node[1] > 0.1 and node[1] < 0.9:
                    WD[5:50, i] = 0
                    WD[75:-1, i] = 0
        WD = WD

        return {
            "v": v,
            "u": u,
            "water_depth": WD,
            "time": t,
            "nodes": nodes,
        }

In [ ]:
Roadmap_t = flow_potentiaalveld(
    nl = (3, 2.5),
    dx_min = 0.1,
    blend = 1,
    vship = np.array([[4]]),
    WD_min = np.array([[5]]),
    ukc = 1.5,
    WWL = 40,
    WVPI = [10000],
    number_of_neighbor_layers = 2,
    optimization_type=["time"],
)

Roadmap_d = flow_dyn_cur(
    nl = (3, 2.5),
    dx_min = 0.1,
    blend = 1,
    vship = np.array([[4]]),
    WD_min = np.array([[5]]),
    ukc = 1.5,
    WWL = 40,
    WVPI = [10000],
    number_of_neighbor_layers = 2,
    optimization_type=["time"],
)

Roadmap = flow_wait(
    nl = (3, 2.5),
    dx_min = 0.1,
    blend = 1,
    vship = np.array([[4]]),
    WD_min = np.array([[5]]),
    ukc = 1.5,
    WWL = 40,
    WVPI = [10000],
    optimization_type=["time", "space"],
    number_of_neighbor_layers = 1
)

In [ ]:
%%time
Roadmap_t.parse()

In [ ]:
%%time
Roadmap_d.parse()

In [ ]:
%%time
Roadmap.parse()

In [ ]:
t0 = "23/03/2019 00:00:00"

path_t, time_t, dist_t = HALEM_time((0.1, 0.1), (0.9, 0.9), t0, 4, Roadmap_t)
path_d, time_d, dist_d = HALEM_time((0.1, 0.1), (0.9, 0.9), t0, 4, Roadmap_d)
path_w, time_w, dist_w = HALEM_space((0.5, 0.1), (0.5, 0.9), t0, 4, Roadmap)

In [ ]:
fig = plt.figure(figsize=(23, 17))

ax = plt.subplot(2, 3, 1)
plt.axis("square")
a = 1

x_r = np.arange(0, 1, 0.075)
y_r = np.arange(0, 1, 0.075)
y_r, x_r = np.meshgrid(y_r, x_r)

WD_r = griddata(
    (Roadmap_t.nodes[:, 1], Roadmap_t.nodes[:, 0]),
    Roadmap_t.WD[:, 0],
    (x_r, y_r),
    method="linear",
)
u_r = griddata(
    (Roadmap_t.nodes[:, 1], Roadmap_t.nodes[:, 0]),
    Roadmap_t.u[:, 0],
    (x_r, y_r),
    method="linear",
)
v_r = griddata(
    (Roadmap_t.nodes[:, 1], Roadmap_t.nodes[:, 0]),
    Roadmap_t.v[:, 0],
    (x_r, y_r),
    method="linear",
)

cval = np.arange(0, 21, 0.5)
im = plt.contourf(x_r, y_r, WD_r, cval)
fig.colorbar(im, ax=ax, label="Waterdepth in meters")

plt.quiver(
    x_r[::a, ::a], y_r[::a, ::a], u_r[::a, ::a], v_r[::a, ::a], label="flow directions"
)
plt.plot(path_t[:, 0], path_t[:, 1], "m", label="Route")
plt.plot(
    Roadmap_t.nodes[:, 1],
    Roadmap_t.nodes[:, 0],
    "k.",
    label="Nodes of the graph",
    markersize=1,
)
plt.plot(path_t[0, 0], path_t[0, 1], "go", label="start")
plt.plot(path_t[-1, 0], path_t[-1, 1], "ro", label="target")


plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title(r"x/y diagram of the simple rotating flow")
ax.legend(
    loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=2, fancybox=True, shadow=True
)
plt.xlabel("lat")
plt.ylabel("lon")

plt.subplot(2, 3, 4)
plot_timeseries(path_t, time_t, Roadmap_t)
plt.title("s/t diagram of the simple rotating flow")

ax = plt.subplot(2, 3, 2)
plt.axis("square")
a = 1

x_r = np.arange(0, 1, 0.075)
y_r = np.arange(0, 1, 0.075)
y_r, x_r = np.meshgrid(y_r, x_r)

WD_r = griddata(
    (Roadmap_d.nodes[:, 1], Roadmap_d.nodes[:, 0]),
    Roadmap_d.WD[:, 0],
    (x_r, y_r),
    method="linear",
)
u_r = griddata(
    (Roadmap_d.nodes[:, 1], Roadmap_d.nodes[:, 0]),
    Roadmap_d.u[:, 0],
    (x_r, y_r),
    method="linear",
)
v_r = griddata(
    (Roadmap_d.nodes[:, 1], Roadmap_d.nodes[:, 0]),
    Roadmap_d.v[:, 0],
    (x_r, y_r),
    method="linear",
)


im = plt.contourf(x_r, y_r, WD_r, cval)
fig.colorbar(im, ax=ax, label="Waterdepth in meters")

plt.quiver(
    x_r[::a, ::a], y_r[::a, ::a], u_r[::a, ::a], v_r[::a, ::a], label="flow directions"
)
plt.plot(path_d[:, 0], path_d[:, 1], "m", label="Route")
plt.plot(
    Roadmap_d.nodes[:, 1],
    Roadmap_d.nodes[:, 0],
    "k.",
    label="Nodes of the graph",
    markersize=0.8,
)
plt.plot(path_d[0, 0], path_d[0, 1], "go", label="start")
plt.plot(path_d[-1, 0], path_d[-1, 1], "ro", label="target")


plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title(r"x/y diagram of the time dependent rotating flow")
ax.legend(
    loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=2, fancybox=True, shadow=True
)
plt.xlabel("lat")
plt.ylabel("lon")

plt.subplot(2, 3, 5)
plot_timeseries(path_d, time_d, Roadmap_d)
plt.title("s/t diagram of the time dependent rotating flow")

ax = plt.subplot(2, 3, 3)
plt.axis("square")
a = 1

x_r = np.arange(0, 1, 0.075)
y_r = np.arange(0, 1, 0.075)
y_r, x_r = np.meshgrid(y_r, x_r)

WD_r = griddata(
    (Roadmap.nodes[:, 1], Roadmap.nodes[:, 0]),
    Roadmap.WD[:, 10],
    (x_r, y_r),
    method="linear",
)
u_r = griddata(
    (Roadmap.nodes[:, 1], Roadmap.nodes[:, 0]),
    Roadmap.u[:, 0],
    (x_r, y_r),
    method="linear",
)
v_r = griddata(
    (Roadmap.nodes[:, 1], Roadmap.nodes[:, 0]),
    Roadmap.v[:, 0],
    (x_r, y_r),
    method="linear",
)

im = plt.contourf(x_r, y_r, WD_r, cval)
fig.colorbar(im, ax=ax, label="Waterdepth in meters")


plt.quiver(
    x_r[::a, ::a], y_r[::a, ::a], u_r[::a, ::a], v_r[::a, ::a], label="flow directions"
)
plt.plot(path_w[:, 0], path_w[:, 1], "m", label="Route")
plt.plot(
    Roadmap.nodes[:, 1],
    Roadmap.nodes[:, 0],
    "k.",
    label="Nodes of the graph",
    markersize=1,
)
plt.plot(path_w[0, 0], path_w[0, 1], "go", label="start")
plt.plot(path_w[-1, 0], path_w[-1, 1], "ro", label="target")


plt.xlim(0, 1)
plt.ylim(0, 1)
plt.title(r"x/y diagram of the flooding and drying flow")
ax.legend(
    loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=2, fancybox=True, shadow=True
)
plt.xlabel("lat")
plt.ylabel("lon")

ax = plt.subplot(2, 3, 6)
plot_timeseries(path_w, time_w, Roadmap)
plt.title("s/t diagram of the flooding and drying flow")


# plt.savefig("D:testcases", dpi=200)
plt.show()